In [1]:
import cv2
import glob
from IPython.display import HTML
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
import numpy as np
import os

In [16]:
def cal_undistort(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

def blur_image(img, kernel_size=(5,5)):
    
    return cv2.GaussianBlur(img, kernel_size, 0)

def get_combined_binary_img(img, saturation_threshold=(180, 215), sobel_x_threshold=(50, 90), hsv_s_threshold=(200, 215)):
   
    img_copy = np.copy(img)
    
    color_channel = img_copy[:,:,2]

    hls_channel = cv2.cvtColor(img_copy, cv2.COLOR_RGB2HLS)[:,:,2]

    hsv_channel = cv2.cvtColor(img_copy, cv2.COLOR_RGB2HSV)[:,:,1]

    yellow_mask = cv2.inRange(img_copy, np.array([150, 150, 0]), np.array([255, 255, 120]))

    white_mask = cv2.inRange(img_copy, np.array([185, 185, 185]), np.array([255, 255, 255]))

    sobelx = cv2.Sobel(color_channel, cv2.CV_64F, 1, 0) 
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sobel_x_threshold[0]) & (scaled_sobel <= sobel_x_threshold[1])] = 1

    hls_s_binary = np.zeros_like(hls_channel)
    hls_s_binary[(hls_channel >= saturation_threshold[0]) & (hls_channel <= saturation_threshold[1])] = 1

    hsv_s_binary = np.zeros_like(hsv_channel)
    hsv_s_binary[(hsv_channel >= hsv_s_treshold[0]) & (hsv_channel <= hsv_s_treshold[1])] = 1

    yellow_binary = (yellow_mask // 255).astype(np.uint8)

    white_binary = (white_mask // 255).astype(np.uint8)

    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(hls_s_binary == 1) | (sxbinary == 1) | (hsv_s_binary == 1) | (yellow_binary == 1) | (white_binary == 1)] = 1
    return combined_binary


def region_of_interest(img, vertices):
    
    mask = np.zeros_like(img)   
    
    ignore_mask_color = (255,)
        
    cv2.fillPoly(mask, np.int32([vertices]), ignore_mask_color)
    
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def apply_perspective_transform(img):
   
    src = np.float32([[560,460],[180,690],[1130,690],[750,460]])
    dst = np.float32([[320,0],[320,720],[960,720],[960,0]])

    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    transformed_img = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    
    return Minv, transformed_img
    
def find_lane_pixels(img, nwindows=9, margin=100, minpix=50):
   
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)

    out_img = np.dstack((img, img, img))
    
    
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    window_height = np.int(img.shape[0]//nwindows)
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(255,0,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(255,0,0), 2) 
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        pass

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    ym_per_pixel = 30/720 
    xm_per_pixel = 3.7/700 
    
    left_fit_m = np.polyfit(lefty*ym_per_pixel, leftx*xm_per_pixel, 2)
    right_fit_m = np.polyfit(righty*ym_per_pixel, rightx*xm_per_pixel, 2)
    
    radius, offset = get_radius_and_offset(left_fit_m, right_fit_m, ym_per_pixel, xm_per_pixel)

    lane_pixel_img = np.dstack((img, img, img))*255
    lane_pixel_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    lane_pixel_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    return leftx, lefty, rightx, righty, radius, offset, out_img

def draw_polynomial(img, fit):
    y = np.linspace(0, img.shape[0]-1, img.shape[0])
    x = fit[0]*y**2 + fit[1]*y + fit[2]
    pts = np.array([np.transpose(np.vstack([x, y]))])
    cv2.polylines(img, np.int_(pts), isClosed=False, color=(0, 255,0), thickness=5)

def fit_polynomial(binary_warped):
    
    leftx, lefty, rightx, righty, radius, offset, out_img = find_lane_pixels(binary_warped)

    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    draw_polynomial(out_img, left_fit)
    draw_polynomial(out_img, right_fit)
    
    return ploty, left_fit, right_fit, radius, offset, out_img

def draw(original_img, img, left_fit, right_fit, Minv):
    
    warp_zero = np.zeros_like(img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    leftx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    rightx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    pts_left = np.array([np.transpose(np.vstack([leftx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([rightx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    cv2.fillPoly(color_warp, np.int_([pts]), (0,0, 255))

    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    result = cv2.addWeighted(original_img, 1, newwarp, 0.3, 0)
    return result

def get_radius_and_offset(left_fit, right_fit, ym_per_pixel, xm_per_pixel):
   
    left_curverad =  ((1 + (2*left_fit[0]*720*ym_per_pixel + left_fit[1])**2)**(3/2))/np.abs(2*left_fit[0])
    right_curverad =  ((1 + (2*right_fit[0]*720*ym_per_pixel + right_fit[1])**2)**(3/2))/np.abs(2*right_fit[0])
    
    left_lane = left_fit[0]*(720*ym_per_pixel)**2 + left_fit[1]*720*ym_per_pixel + left_fit[2]
    right_lane = right_fit[0]*(720*ym_per_pixel)**2 + right_fit[1]*720*ym_per_pixel + right_fit[2]
    
    radius = np.mean([left_curverad, right_curverad])
    offset = [640*xm_per_pixel - np.mean([left_lane, right_lane]), right_lane-left_lane]
    return radius, offset

def add_calculations(result_img, radius, offset):   
   
    cv2.putText(result_img, 'Egri yarıcapi: {} m'.format(round(radius)), (50, 200), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(result_img, 'BTU Elektronomi', (50, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 4)
    return result_img

In [17]:
class PipelineVideo(object):
    def __init__(self):
        self.frame_ind = 0
        self.left_fit = None
        self.right_fit = None
        self.radius = None
        self.offset = None
        self.frames = {}
    
    def update_radius_delta(self, radius):
        if (self.frame_ind == 0) | (self.radius is None):
            return 0
        else:
            return abs((radius-self.radius)/self.radius)
        
    def update_offset_delta(self, offset):
        if (self.frame_ind == 0) | (self.offset is None):
            return 0
        else:
            offset[0]
            return abs((offset[0]-self.offset[0])/self.offset[0])

In [18]:
saturation_threshold = (180, 200)
sobel_x_threshold = (40, 100)
hsv_s_treshold = (200, 215)

objpoints = []
imgpoints = []

chessboard_shape = (9, 6)
objp = np.zeros((chessboard_shape[0]*chessboard_shape[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboard_shape[0], 0:chessboard_shape[1]].T.reshape(-1, 2)

for filename in glob.glob('./camera_cal/*.jpg'):    
    img = mpimg.imread(filename)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, chessboard_shape, None)
    
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)

def image_pipeline(img, pipeline):
    img_copy = np.copy(img)
    
    distortion_corrected_img = cv2.undistort(img_copy, mtx, dist, None, mtx)

    combined_binary_img = get_combined_binary_img(distortion_corrected_img)
    
    vertices = np.array([[150, combined_binary_img.shape[0]], 
                         [580, combined_binary_img.shape[0]*0.6], 
                         [745, combined_binary_img.shape[0]*0.6], 
                         [1150, combined_binary_img.shape[0]]], np.int32)
    roi_img = region_of_interest(combined_binary_img, [vertices])

    Minv, binary_warped = apply_perspective_transform(roi_img)

    ploty, left_fit, right_fit, radius, offset, out_img = fit_polynomial(binary_warped)
    
    pipeline.frames[pipeline.frame_ind] = {
        'left_fit': left_fit,
        'right_fit': right_fit,
        'radius': radius,
        'radius_delta': pipeline.update_radius_delta(radius),
        'offset': offset,
        'offset_delta': pipeline.update_offset_delta(offset),
        'updated': True
    }

    if pipeline.frame_ind == 0:
        pipeline.offset = offset
        pipeline.radius = radius
        
    if ((abs((offset[0]-pipeline.offset[0])/pipeline.offset[0]) > 5) | \
       ((pipeline.radius < 1000) & (abs((radius-pipeline.radius)/pipeline.radius) > 30))) & \
       (pipeline.frame_ind > 0):
        pipeline.frames[pipeline.frame_ind]['updated'] = False
        left_fit = pipeline.left_fit
        right_fit = pipeline.right_fit
        radius = pipeline.radius
        offset = pipeline.offset

    projected_lanes_img = draw(img_copy, binary_warped, left_fit, right_fit, Minv)

    out_img = add_calculations(projected_lanes_img, radius, offset)
    
    pipeline.frame_ind += 1
    pipeline.left_fit = left_fit
    pipeline.right_fit = right_fit
    pipeline.radius = radius
    pipeline.offset = offset

    return out_img

In [19]:
pipeline = PipelineVideo()
video_out = './output_video/lane_detection_video.mp4'

video_clip = VideoFileClip("project_video.mp4")
clip = video_clip.fl_image(lambda x: image_pipeline(x, pipeline))
%time clip.write_videofile(video_out, audio=False)

t:   0%|                                                                    | 2/1260 [00:00<01:08, 18.39it/s, now=None]

Moviepy - Building video ./output_video/lane_detection_video.mp4.
Moviepy - Writing video ./output_video/lane_detection_video.mp4



Moviepy - Done !
Moviepy - video ready ./output_video/lane_detection_video.mp4
Wall time: 2min 11s


In [20]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_out))